In [1]:
import pandas as pd
from pathlib import Path
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
import warnings

In [10]:
df_train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv',
                      index_col = 'id',
                      #nrows = 25000
                      )
features = df_train.columns[:-1] 
target = df_train.columns[-1]

df_train.head()


,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
id,,,,,,,,,,,,,,,,,,,,,
0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,86.489,...,-12.2280,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.1,1.01770,1
1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,9953.600,...,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.0,0.28359,0
2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,15827.000,...,-5.7688,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.2,0.40690,1
3,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,-36.837,...,-34.8580,2.0694,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.1,0.51486,1
4,0.11623,0.502900,-109.150,0.297910,0.34490,-0.40932,2538.90,65332.0,1.907200e+15,144.120,...,-13.6410,1.5298,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,NaN,0.23049,1


In [13]:
train = df_train.loc[:, features]
y = df_train.loc[:, target]

model = XGBClassifier(max_depth=3, 
                     subsample=0.5,
                     colsample_bytree=0.5,
                     n_jobs=-1,
                     tree_method='gpu_hist',
                     random_state=0)
def score(X, y, model, cv):
    scoring = ['roc_auc'] #AUC (area under curve) evaluation metric
    scores = cross_validate(model, X, y,
                           scoring=scoring,
                           cv=cv, return_train_score=True
                           )
    scores = pd.DataFrame(scores).T
    return scores.assign(mean=lambda x: x.mean(axis=1),
                        std=lambda x: x.std(axis=1),
                        )
scores = score(train, y, model, cv = 3)
display(scores)

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:51:30] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:51:35] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:51:41] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,0,1,2,mean,std
fit_time,2.881614,3.227886,3.327554,3.145685,0.191108
score_time,0.829991,0.800690,0.787539,0.806073,0.017744
test_roc_auc,0.798027,0.796957,0.797926,0.797637,0.000482
train_roc_auc,0.799801,0.800660,0.800500,0.800320,0.000373


In [17]:
model.fit(train, y)
xtest = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv',
                    index_col='id')

y_pred = pd.Series(model.predict(xtest),
                 index=xtest.index,
                 name=target,
                 )
y_pred.to_csv('submission.csv')

[06:54:22] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
